In [1]:
import warnings
import math

import pandas as pd

from jre_utils.datapath import (
    factor_data_paths,
    model_ready_data_paths,
    get_derived_csv_path,
    get_derived_lpa_path,
    get_derived_plps_path,
)

from jre_utils.process import get_most_active_municipalities

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
asset_type = "building"
years_ahead = 2

metrics = {
    "weighted_mean": "unit_price_wmean",
    "weighted_median": "unit_price_wmedian",
    "mean": "unit_price_mean",
    "median": "unit_price_median",
    "weighted_mean_smoothed": "unit_price_wmean_smoothed",
    "weighted_median_smoothed": "unit_price_wmedian_smoothed",
    "mean_smoothed": "unit_price_mean_smoothed",
    "median_smoothed": "unit_price_median_smoothed",
}

dataset_paths = {
    "transactions": get_derived_csv_path(asset_type),
    "lpa": get_derived_lpa_path(),
    "plps": get_derived_plps_path()
}

granularity_columns = ["area", "area_code"]
group_by_columns = granularity_columns + ["year"]
display_columns = ["unit_price", "total_traded_area", "count"]

metric_key = "weighted_median_smoothed"
metric = metrics[metric_key]
metric_pct_chg = metric + "_pct_chg"
upcoming_metric = "upcoming_" + metric

In [3]:
dataset_key = "transactions"
core_path = dataset_paths[dataset_key]
population_path = factor_data_paths["processed"]["population"]["municipality"]
migration_path = factor_data_paths["processed"]["migration"]["municipality"]
taxable_income_path = factor_data_paths["processed"]["taxable_income"]["municipality"]
new_dwellings_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]
lfs_revenue_path = factor_data_paths["processed"]["lfs_revenue_breakdown"]["municipality"]


In [4]:
def years_since_crisis(year):
    year_ranges = {
        (1960, 1973): 1960,
        (1973, 1990): 1973,
        (1990, 1997): 1990,
        (1997, 2008): 1997,
        (2008, 2019): 2008,
        (2019, math.inf): 2019,
    }

    for range_start, range_end in year_ranges:
        if range_start <= year < range_end:
            return year - year_ranges[(range_start, range_end)]
        

In [5]:
df = pd.read_csv(core_path)
df = get_most_active_municipalities(df, 1500)

population_df = pd.read_csv(population_path)
migration_df = pd.read_csv(migration_path)
taxable_income_df = pd.read_csv(taxable_income_path)
new_dwellings_df = pd.read_csv(new_dwellings_path)
lfs_revenue_df= pd.read_csv(lfs_revenue_path)

df = (
    df.merge(population_df, on=group_by_columns, how="left")
    .merge(migration_df, on=group_by_columns, how="left")
    .merge(taxable_income_df, on=group_by_columns, how="left")
    .merge(new_dwellings_df, on=group_by_columns, how="left")
    .merge(lfs_revenue_df, on=group_by_columns, how="left")
)

In [6]:
df["migrations_is_available"] = df["net_migration_ratio"].notnull().astype(int)
df["taxable_income_is_available"] = df["taxable_income"].notnull().astype(int)
df["total_tax_is_available"] = df["total_tax"].notnull().astype(int)

# Might go back and undo the new dwellings filling for unknown municipalities.
df["dwellings_is_available"] = df["new_dwellings"].notnull().astype(int)

# Years since crisis (Yn - 2008) or (Yn - 2020)
df["years_since_crisis"] = df["year"].apply(years_since_crisis)

df = df.fillna(0)
# df

In [7]:
df[df["area_code"] == 22205]

,year,area_code,area,unit_price_wmean,unit_price_wmedian,unit_price_mean,unit_price_median,total_traded_area,count,unit_price_wmean_smoothed,unit_price_wmedian_smoothed,unit_price_mean_smoothed,unit_price_median_smoothed,population,net_migration_ratio,taxable_income,taxpayer_count,taxable_income_per_taxpayer,taxable_income_growth,taxable_income_per_taxpayer_growth,new_dwellings,existing_dwellings,new_dwellings_ratio,total_tax,total_tax_growth,migrations_is_available,taxable_income_is_available,total_tax_is_available,dwellings_is_available,years_since_crisis
3506,2007,22205,Shizuoka-ken Atami-shi,143886.604631,80112.280702,101095.409005,74835.309618,23105.0,56.0,138921.746893,78807.061329,100054.880742,73068.957809,32987.0,-0.001273,60474545.0,19270.0,3138.274261,0.055514,0.054363,1717.0,34678.0,0.049513,11067092.0,0.023212,1,1,1,1,10
3507,2008,22205,Shizuoka-ken Atami-shi,270118.973075,61660.325513,128146.418278,59615.384615,23955.0,55.0,102893.414491,63920.151612,86023.498267,61006.858874,32945.0,0.000789,62851130.0,19054.0,3298.579301,0.039299,0.051081,546.0,36395.0,0.015002,11011410.0,-0.005031,1,1,1,1,0
3508,2009,22205,Shizuoka-ken Atami-shi,44118.488253,25432.134879,61331.538646,43910.658307,24475.0,66.0,44157.122475,48804.375141,64473.842035,49815.628472,32971.0,0.000607,57521629.0,18858.0,3050.250769,-0.084796,-0.075283,117.0,36941.0,0.003167,10489161.0,-0.047428,1,1,1,1,1
3509,2010,22205,Shizuoka-ken Atami-shi,81377.401778,35947.712418,73227.794077,47619.047619,34870.0,81.0,62367.538775,35947.712418,65675.875675,44287.325734,32991.0,0.002425,52691475.0,18391.0,2865.068512,-0.083971,-0.060711,328.0,37058.0,0.008851,10502168.0,0.001240,1,1,1,1,2
3510,2011,22205,Shizuoka-ken Atami-shi,53718.244804,28225.308642,57165.025170,39393.939394,17320.0,51.0,59069.101902,31430.709603,64726.576827,42315.231503,33071.0,0.004929,50732226.0,17776.0,2853.973110,-0.037183,-0.003873,341.0,37386.0,0.009121,10226135.0,-0.026283,1,1,1,1,3
3511,2012,22205,Shizuoka-ken Atami-shi,44935.382466,31272.395097,67438.915375,41338.235294,28630.0,80.0,46735.641010,29482.359962,60011.571406,39689.138088,33234.0,0.004453,48997548.0,17216.0,2846.047165,-0.034193,-0.002777,101.0,37727.0,0.002677,9791024.0,-0.042549,1,1,1,1,4
3512,2013,22205,Shizuoka-ken Atami-shi,42447.726276,27936.484121,52015.798695,37500.000000,34195.0,77.0,44735.169975,30162.666029,55845.651265,36840.781837,33382.0,0.003475,48967696.0,17136.0,2857.591970,-0.000609,0.004056,110.0,37828.0,0.002908,9793374.0,0.000240,1,1,1,1,5
3513,2014,22205,Shizuoka-ken Atami-shi,48557.745441,32299.325980,50484.300968,31250.000000,27145.0,67.0,85883.986373,33905.109157,61371.017374,37349.090175,33498.0,-0.000478,50435446.0,17165.0,2938.272415,0.029974,0.028234,384.0,37938.0,0.010122,9967010.0,0.017730,1,1,1,1,6
3514,2015,22205,Shizuoka-ken Atami-shi,186328.467153,42435.643564,85293.219545,44954.954955,27400.0,68.0,132835.179559,39018.252360,79129.715136,41428.263127,33482.0,0.004510,50358761.0,16940.0,2972.772196,-0.001520,0.011742,142.0,38322.0,0.003705,9762576.0,-0.020511,1,1,1,1,7
3515,2016,22205,Shizuoka-ken Atami-shi,161778.481013,41100.713012,97033.810906,46130.952381,28440.0,74.0,140947.330581,43087.769156,82502.341735,46875.665370,33633.0,0.004936,51725102.0,17044.0,3034.798287,0.027132,0.020865,63.0,38464.0,0.001638,9757556.0,-0.000514,1,1,1,1,8


In [7]:
# prepare metrics
df = df.sort_values(by=group_by_columns, ascending=True)
df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change(periods=years_ahead)

# time box
# start_year = 2005
# end_year = 2023

# df = df[(df["year"] >= start_year) & (df["year"] <= end_year)]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25315 entries, 16845 to 14370
Data columns (total 31 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   year                                 25315 non-null  int64  
 1   area_code                            25315 non-null  int64  
 2   area                                 25315 non-null  object 
 3   unit_price_wmean                     25315 non-null  float64
 4   unit_price_wmedian                   25315 non-null  float64
 5   unit_price_mean                      25315 non-null  float64
 6   unit_price_median                    25315 non-null  float64
 7   total_traded_area                    25315 non-null  float64
 8   count                                25315 non-null  float64
 9   unit_price_wmean_smoothed            25315 non-null  float64
 10  unit_price_wmedian_smoothed          25315 non-null  float64
 11  unit_price_mean_smoothed     

In [9]:
df.columns

Index(['year', 'area_code', 'area', 'unit_price_wmean', 'unit_price_wmedian',
       'unit_price_mean', 'unit_price_median', 'total_traded_area', 'count',
       'unit_price_wmean_smoothed', 'unit_price_wmedian_smoothed',
       'unit_price_mean_smoothed', 'unit_price_median_smoothed', 'population',
       'net_migration_ratio', 'taxable_income', 'taxpayer_count',
       'taxable_income_per_taxpayer', 'taxable_income_growth',
       'taxable_income_per_taxpayer_growth', 'new_dwellings',
       'existing_dwellings', 'new_dwellings_ratio', 'total_tax',
       'total_tax_growth', 'migrations_is_available',
       'taxable_income_is_available', 'total_tax_is_available',
       'dwellings_is_available', 'years_since_crisis',
       'unit_price_wmedian_smoothed_pct_chg'],
      dtype='object')

In [10]:
columns = [
    metric_pct_chg,
    metric,
    "year",
    "years_since_crisis",
    "count",
    "total_traded_area",
    "population",
    "taxpayer_count",
    "taxable_income",
    "taxable_income_per_taxpayer",
    "taxable_income_growth",
    "taxable_income_per_taxpayer_growth",
    "total_tax",
    "total_tax_growth",
    "new_dwellings",
    "existing_dwellings",
    "net_migration_ratio",
    "new_dwellings_ratio",
    "migrations_is_available",
    "taxable_income_is_available",
    "dwellings_is_available",
    "total_tax_is_available",
    "area_code",  # id
    "area", # additional info
]

# add ratios and growths if necessary

df = df[columns]
df = df.dropna()

In [11]:
print(f"Initial Size: ", df.shape[0])
q = 0.01
filtered_df = df.copy()
filter_col = metric_pct_chg
filtered_df = filtered_df[
    (filtered_df[filter_col] >= filtered_df[filter_col].quantile(q))
    & (filtered_df[filter_col] <= filtered_df[filter_col].quantile(1 - q))
]
print(f"Filtered Size: ", filtered_df.shape[0])
filtered_df.describe()

Initial Size:  22315
Filtered Size:  21867


,unit_price_wmedian_smoothed_pct_chg,unit_price_wmedian_smoothed,year,years_since_crisis,count,total_traded_area,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,taxable_income_growth,taxable_income_per_taxpayer_growth,total_tax,total_tax_growth,new_dwellings,existing_dwellings,net_migration_ratio,new_dwellings_ratio,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available,area_code
count,21867.000000,2.186700e+04,21867.000000,21867.000000,21867.000000,21867.000000,2.186700e+04,2.186700e+04,2.186700e+04,21867.000000,21867.000000,21867.000000,2.186700e+04,21867.000000,21867.000000,2.186700e+04,21867.000000,21867.000000,21867.000000,21867.000000,21867.000000,21867.000000,21867.000000
mean,-0.000593,7.208540e+04,2015.890794,4.354415,78.132803,20714.112818,8.304147e+04,3.521177e+04,1.181324e+08,2522.167423,0.002475,0.002178,1.289725e+07,0.003247,531.916770,3.423459e+04,-0.003072,0.006383,0.937852,0.873417,0.488636,0.940047,21467.543559
std,0.273908,1.962551e+05,4.321446,3.028851,198.863859,39885.026520,2.055271e+05,9.243731e+04,3.471286e+08,1095.047512,0.041031,0.031123,4.052008e+07,0.061631,1794.725477,1.016387e+05,0.006989,0.007971,0.241430,0.332513,0.499882,0.237406,13798.652884
min,-0.621123,1.025209e+03,2007.000000,0.000000,1.000000,65.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.537047,-0.480760,0.000000e+00,-0.677251,0.000000,0.000000e+00,-0.313065,0.000000,0.000000,0.000000,0.000000,0.000000,1100.000000
25%,-0.167355,1.365116e+04,2012.000000,2.000000,9.000000,3907.500000,1.112650e+04,3.557500e+03,9.233596e+06,2457.305510,-0.008292,-0.006307,1.372152e+06,-0.014501,0.000000,0.000000e+00,-0.006570,0.000000,1.000000,1.000000,0.000000,1.000000,10344.000000
50%,-0.027596,2.676741e+04,2016.000000,4.000000,24.000000,9375.000000,2.992700e+04,1.130500e+04,3.040271e+07,2718.857673,0.001458,0.000204,3.749085e+06,0.000000,0.000000,0.000000e+00,-0.002818,0.000000,1.000000,1.000000,0.000000,1.000000,20521.000000
75%,0.119323,6.694880e+04,2020.000000,7.000000,70.000000,21392.500000,7.357050e+04,2.991050e+04,8.984681e+07,3028.263334,0.018424,0.011466,1.038226e+07,0.017280,384.000000,2.970550e+04,0.000000,0.012158,1.000000,1.000000,1.000000,1.000000,32449.000000
max,1.272794,6.241887e+06,2023.000000,10.000000,4358.000000,666150.000000,3.794024e+06,1.906224e+06,7.965148e+09,12667.020000,1.066040,1.026488,8.672765e+08,4.759148,39143.000000,1.916062e+06,0.099253,0.120270,1.000000,1.000000,1.000000,1.000000,47362.000000


In [12]:
filtered_df.to_csv(model_ready_data_paths[f"sequence_{dataset_key}_{metric_key}_{years_ahead}"], index=False)

In [13]:
filtered_df[filtered_df["area_code"] == 13101][["year", metric, metric_pct_chg]]

,year,unit_price_wmedian_smoothed,unit_price_wmedian_smoothed_pct_chg
2012,2007,3.190579e+06,0.121657
2013,2008,3.144827e+06,0.055103
2014,2009,2.910509e+06,-0.087780
2015,2010,2.697234e+06,-0.142327
2016,2011,2.354779e+06,-0.190939
2017,2012,2.224987e+06,-0.175086
2018,2013,2.791684e+06,0.185540
2019,2014,3.167834e+06,0.423754
2020,2015,3.297759e+06,0.181280
2021,2016,3.560900e+06,0.124081
